### Requirements
1. GPUインスタンスで立ち上げを行う
2. Google Driveをマウントする

### 必要なライブラリをインストール

In [1]:
!pip install transformers fugashi ipadic

     |████████████████████████████████| 1.1MB 6.5MB/s 
     |████████████████████████████████| 481kB 22.7MB/s 
     |████████████████████████████████| 13.4MB 238kB/s 
     |████████████████████████████████| 3.0MB 39.8MB/s 
     |████████████████████████████████| 890kB 36.0MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp36-none-any.whl size=13556725 sha256=a18a9e8aa0cb1e9da0037402fa8ce82c66045fbdb05ebb47d59ecad6fee051c7
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c95fbdee9a3c1e89421f11305159622b4b924a273b4eb27033f4ee3fbdc0f99a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built ipadic sacremoses


### カレントディレクトリに移動
- レポジトリ内の`bert`ディレクトリに移動

In [2]:
%cd /content/drive/My Drive/research/brigade-visualizer/source_code/bert

/content/drive/My Drive/research/brigade-visualizer/source_code/bert


### アンケート結果を読み込む

In [3]:
import pathlib
import os
import pandas as pd
import torch

In [4]:
brigade_data_path = pathlib.Path("./brigade_enq.csv")
brigade_data_path.exists()

True

In [5]:
df = pd.read_csv(brigade_data_path)
display(df.head(2))

,Timestamp,あなたのブリゲード名をご記入下さい。,取り組んでいる地域課題や興味分野を選んでください（複数選択可能）,取り組んでいる技術などを選んでください（複数選択可能）,"もし他にもブリゲードの特徴を表すキーワードなどあればご記入下さい（複数ある場合は半角カンマ「,」で区切って下さい）",ブリゲードでの主な活動を教えてください（複数選択可）,コンテストやイベントへの出展について、応募経験や検討をしたものがあれば選んでください（複数選択可能）,ブリゲードの設立年月日を教えてください。,ブリゲードでの活動は概ね何人くらいで行われていますか？一番近いものを選択してください。,ブリゲードメンバーの属性を教えてください。,活動の頻度を教えてください（すべての活動での平均を教えてください）,もしミッションステートメントや活動上のモットー等ありましたらご記入下さい。無い場合は、なにも記入せずに進んでください。,もしよろしければブリゲードの紹介文をご記入下さい。無い場合は、なにも記入せずに進んでください。
0,7/1/2020 21:00:39,CODE for GIFU,"COVID-19, 異文化交流, 河川, 健康, 福祉, まちづくり, 林業","GIS・位置空間情報, オープンデータ, プログラミング",手芸部や天文部がある,"飲み会・食事会・交流会, 勉強会・イベント運営・企画, 他の団体のイベントへの参加, 他の団...","FOSS4G, MA ヒーローズリーグ, RESASアプリコンテスト, アーバンデータチャレ...",11/1/2013,5人〜10人,"ITエンジニア, IT以外のエンジニア, デザイナー, プランナー・コンサルタント, 会社員...",月に2〜4回,テクノロジーを使って身の回りの困りごとを自分達の手で解決する,「テクノロジーなんてよくわからない」「自分でも出来るかな？」という方にも参加して欲しい。何故...
1,7/2/2020 17:03:55,Code for Saga,"COVID-19, SDGs, 交通, 地域アーカイブ, 防災, まちづくり","GIS・位置空間情報, オープンデータ, グラレコ・ファシリテーション, ノーコード, プロ...","コミュニティ,ネットワーキング","飲み会・食事会・交流会, 定例の打ち合わせ, プロダクト開発・運営, 勉強会・イベント運営・...","アーバンデータチャレンジ, 佐賀大学Dフェス",2/22/2014,10人〜20人,"ITエンジニア, デザイナー, プランナー・コンサルタント, 会社員・団体職員（上記以外）,...",月に1回,ICTの力で、私たちの街をもっと暮らしやすいところに…,コードが書ける人も書けない人も、ICTが得意な人もそうでない人も、みんなでワイワイと楽しみな...


### BERTモデルを読み込む
- 東北大学 乾・鈴木研究室が公開している[訓練済み日本語BERTモデル](https://www.nlp.ecei.tohoku.ac.jp/news-release/3284/)を使用
  - https://github.com/cl-tohoku/bert-japanese

In [6]:
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers.modeling_bert import BertModel

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', )

### 文のベクトルを取得する
- 最後から2つ目の隠れ層を取得し平均したものを文章の埋め込みベクトルとする
- bert-as-serviceを参考
  - https://github.com/hanxiao/bert-as-service#q-how-do-you-get-the-fixed-representation-did-you-do-pooling-or-something
  

In [7]:
def sentence_representation(text):
  encoded = tokenizer.encode_plus(text, return_tensors='pt')
  encoded["return_dict"] = True
  encoded["output_hidden_states"] = True

  result = model(**encoded)
  return torch.mean(result.hidden_states[-2], 1)

In [8]:
cos = torch.nn.CosineSimilarity(dim=1)

In [13]:
from collections import OrderedDict

brigade_rep_dic = OrderedDict()
brigade_keyword_rep_dic = OrderedDict()

brigade_keyword_distance_dic = {
    "brigade": [],
    "keyword": [],
    "distance": []
}

for row in df.itertuples():
  brigade_name = row[2]
  brigade_keywords = [keyword.strip() for keyword in row[3].split(',') + row[4].split(',') + row[7].split(',')]

  brigade_rep = torch.zeros(1, 768)

  for keyword in brigade_keywords:
    if keyword in brigade_keyword_rep_dic:
      keyword_rep = brigade_keyword_rep_dic[keyword]
    else:
      keyword_rep = sentence_representation(keyword)
      brigade_keyword_rep_dic[keyword] = keyword_rep

    brigade_rep += keyword_rep

  brigade_rep_dic[brigade_name] = brigade_rep

  for keyword in brigade_keywords:
    brigade_keyword_distance_dic["brigade"].append(brigade_name)
    brigade_keyword_distance_dic["keyword"].append(keyword)

    distance = cos(brigade_rep, brigade_keyword_rep_dic[keyword])
    brigade_keyword_distance_dic["distance"].append(distance.detach().numpy().copy()[0])

In [14]:
df_brigade_keyword_distance = pd.DataFrame.from_dict(brigade_keyword_distance_dic)
display(df_brigade_keyword_distance)

,brigade,keyword,distance
0,CODE for GIFU,COVID-19,0.923896
1,CODE for GIFU,異文化交流,0.958433
2,CODE for GIFU,河川,0.950639
3,CODE for GIFU,健康,0.977177
4,CODE for GIFU,福祉,0.964413
...,...,...,...
517,Code for SUSONO,オープンデータ,0.966124
518,Code for SUSONO,ノーコード,0.948447
519,Code for SUSONO,プログラミング,0.973487
520,Code for SUSONO,アーバンデータチャレンジ,0.963252


In [43]:
import json

text_list = []
for row in df_brigade_keyword_distance.itertuples():
  text = row[1] + "," + row[2] + "," +  str(row[3])
  text_list.append(text)

with open("../json/brigade-keyword-distance-bert.json", 'w') as f:
  json.dump(text_list, f, ensure_ascii=False, indent=3)

In [30]:
brigade_distance_dic = {
    "brigade1": [],
    "brigade2": [],
    "distance": []
}


brigade_rep_list = list(brigade_rep_dic.items())

for brigade1_idx, (brigade1_name, brigade1_sentence_rep) in enumerate(brigade_rep_list):
  for brigade2_idx, (brigade2_name, brigade2_sentence_rep) in enumerate(brigade_rep_list[brigade1_idx+1:]):
    brigade_distance_dic["brigade1"].append(brigade1_name)
    brigade_distance_dic["brigade2"].append(brigade2_name)

    distance = cos(brigade1_sentence_rep, brigade2_sentence_rep)
    brigade_distance_dic["distance"].append(distance.detach().numpy().copy()[0])

In [31]:
df_brigade_distance = pd.DataFrame.from_dict(brigade_distance_dic)
display(df_brigade_distance)

,brigade1,brigade2,distance
0,CODE for GIFU,Code for Saga,0.995224
1,CODE for GIFU,Code for NAGAREYAMA,0.992532
2,CODE for GIFU,Code for Nagoya,0.997259
3,CODE for GIFU,Code for SAITAMA,0.997025
4,CODE for GIFU,CODE for AIZU,0.997812
...,...,...,...
625,Code for Shinagawa,Code for Muroran,0.987470
626,Code for Shinagawa,Code for SUSONO,0.994133
627,Code for 中野,Code for Muroran,0.991699
628,Code for 中野,Code for SUSONO,0.996944


In [44]:
import json

text_list = []
for row in df_brigade_distance.itertuples():
  text = row[1] + "," + row[2] + "," +  str(row[3])
  text_list.append(text)

with open("../json/brigade-distance-bert.json", 'w') as f:
  json.dump(text_list, f, ensure_ascii=False, indent=3)